In [1]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
from sklearn.linear_model import LinearRegression
warnings.filterwarnings('ignore')

import pickle
import os
from statsmodels.formula.api import ols

In [2]:
im_db = 'data/im.db'

im_db_conn = sqlite3.connect(im_db)

In [3]:
q = """

SELECT *
FROM sqlite_master


;
"""

pd.read_sql(q, im_db_conn)

,type,name,tbl_name,rootpage,sql
0,table,movie_basics,movie_basics,2,"CREATE TABLE ""movie_basics"" (\n""movie_id"" TEXT..."
1,table,directors,directors,3,"CREATE TABLE ""directors"" (\n""movie_id"" TEXT,\n..."
2,table,known_for,known_for,4,"CREATE TABLE ""known_for"" (\n""person_id"" TEXT,\..."
3,table,movie_akas,movie_akas,5,"CREATE TABLE ""movie_akas"" (\n""movie_id"" TEXT,\..."
4,table,movie_ratings,movie_ratings,6,"CREATE TABLE ""movie_ratings"" (\n""movie_id"" TEX..."
5,table,persons,persons,7,"CREATE TABLE ""persons"" (\n""person_id"" TEXT,\n ..."
6,table,principals,principals,8,"CREATE TABLE ""principals"" (\n""movie_id"" TEXT,\..."
7,table,writers,writers,9,"CREATE TABLE ""writers"" (\n""movie_id"" TEXT,\n ..."


In [4]:
q = """

SELECT DISTINCT *
FROM movie_basics
LEFT JOIN directors
    USING(movie_id)
LEFT JOIN movie_ratings
    USING(movie_id)
LEFT JOIN persons
    USING(person_id)
WHERE numvotes > 9999 and averagerating != 'NaN'
;
"""

cleaned_movies_df = pd.read_sql(q, im_db_conn)

In [5]:
cleaned_movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011 entries, 0 to 3010
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            3011 non-null   object 
 1   primary_title       3011 non-null   object 
 2   original_title      3011 non-null   object 
 3   start_year          3011 non-null   int64  
 4   runtime_minutes     3011 non-null   float64
 5   genres              3011 non-null   object 
 6   person_id           3010 non-null   object 
 7   averagerating       3011 non-null   float64
 8   numvotes            3011 non-null   int64  
 9   primary_name        3010 non-null   object 
 10  birth_year          2070 non-null   float64
 11  death_year          23 non-null     float64
 12  primary_profession  3010 non-null   object 
dtypes: float64(4), int64(2), object(7)
memory usage: 305.9+ KB


In [6]:
cleaned_movies_df.head(30)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm0001317,4.2,50352,Renny Harlin,1959.0,NaN,"producer,director,writer"
1,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",nm0587955,7.6,326657,Bennett Miller,1966.0,NaN,"director,producer,cinematographer"
2,tt1212419,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",nm0000142,6.5,87288,Clint Eastwood,1930.0,NaN,"actor,producer,director"
3,tt1229238,Mission: Impossible - Ghost Protocol,Mission: Impossible - Ghost Protocol,2011,132.0,"Action,Adventure,Thriller",nm0083348,7.4,428142,Brad Bird,1957.0,NaN,"miscellaneous,writer,director"
4,tt1232829,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",nm0520488,7.2,477771,Phil Lord,1975.0,NaN,"producer,writer,director"
5,tt1232829,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",nm0588087,7.2,477771,Christopher Miller,1975.0,NaN,"producer,writer,director"
6,tt1243974,Aloha,Aloha,2015,105.0,"Comedy,Drama,Romance",nm0001081,5.4,54646,Cameron Crowe,1957.0,NaN,"writer,producer,director"
7,tt1261945,Sex and the City 2,Sex and the City 2,2010,146.0,"Comedy,Drama,Romance",nm0455078,4.4,69199,Michael Patrick King,1954.0,NaN,"producer,writer,director"
8,tt1270761,Don't Be Afraid of the Dark,Don't Be Afraid of the Dark,2010,99.0,"Fantasy,Horror,Thriller",nm2552536,5.6,44490,Troy Nixey,NaN,NaN,"director,writer,producer"
9,tt1320239,Burke and Hare,Burke and Hare,2010,91.0,"Comedy,Crime,History",nm0000484,6.2,18431,John Landis,1950.0,NaN,"producer,director,actor"


In [7]:
series = cleaned_movies_df.groupby('primary_name')['averagerating'].mean()

In [8]:
ratings_df = pd.DataFrame(series) 

In [9]:
ratings_df = ratings_df.rename(columns={'averagerating':'director_rating'})

In [10]:
ratings_df = ratings_df.reset_index()
ratings_df.head()

,primary_name,director_rating
0,A.R. Murugadoss,7.600
1,Aanand L. Rai,6.925
2,Aaron Hann,6.000
3,Aaron Harvey,4.600
4,Aaron Horvath,6.800


In [11]:
cleaned_movies_ratings_df = cleaned_movies_df.merge(ratings_df, how='left', on='primary_name')

In [12]:
cleaned_movies_ratings_df.loc[cleaned_movies_ratings_df['primary_name'] == 'Christopher Nolan']

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession,director_rating
413,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller",nm0634240,8.4,1387769,Christopher Nolan,1970.0,NaN,"writer,producer,director",8.425
1063,tt0816692,Interstellar,Interstellar,2014,169.0,"Adventure,Drama,Sci-Fi",nm0634240,8.6,1299334,Christopher Nolan,1970.0,NaN,"writer,producer,director",8.425
2024,tt5013056,Dunkirk,Dunkirk,2017,106.0,"Action,Drama,History",nm0634240,7.9,466580,Christopher Nolan,1970.0,NaN,"writer,producer,director",8.425
2629,tt1375666,Inception,Inception,2010,148.0,"Action,Adventure,Sci-Fi",nm0634240,8.8,1841066,Christopher Nolan,1970.0,NaN,"writer,producer,director",8.425


In [13]:
nummovies_series = cleaned_movies_ratings_df['primary_name'].value_counts()
nummovies_df = pd.DataFrame(nummovies_series)

In [14]:
nummovies_df = nummovies_df.reset_index()

In [15]:
nummovies_df

,index,primary_name
0,Woody Allen,8
1,Steven Spielberg,7
2,Ridley Scott,7
3,Adam Wingard,7
4,Clint Eastwood,7
...,...,...
1883,Rupert Goold,1
1884,Patty Jenkins,1
1885,Jan Ole Gerster,1
1886,Stuart Zicherman,1


In [16]:
nummovies_df = nummovies_df.rename(columns={'index':'primary_name', 'primary_name': 'nummovies'})

In [17]:
cleaned_movies_ratings_df = cleaned_movies_ratings_df.merge(nummovies_df, how='left', on='primary_name')

In [18]:
cleaned_movies_ratings_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession,director_rating,nummovies
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm0001317,4.2,50352,Renny Harlin,1959.0,NaN,"producer,director,writer",5.300000,4.0
1,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",nm0587955,7.6,326657,Bennett Miller,1966.0,NaN,"director,producer,cinematographer",7.300000,2.0
2,tt1212419,Hereafter,Hereafter,2010,129.0,"Drama,Fantasy,Romance",nm0000142,6.5,87288,Clint Eastwood,1930.0,NaN,"actor,producer,director",6.714286,7.0
3,tt1229238,Mission: Impossible - Ghost Protocol,Mission: Impossible - Ghost Protocol,2011,132.0,"Action,Adventure,Thriller",nm0083348,7.4,428142,Brad Bird,1957.0,NaN,"miscellaneous,writer,director",7.166667,3.0
4,tt1232829,21 Jump Street,21 Jump Street,2012,109.0,"Action,Comedy,Crime",nm0520488,7.2,477771,Phil Lord,1975.0,NaN,"producer,writer,director",7.333333,3.0


In [19]:
cleaned_movies_ratings_df.rename(columns= {'primary_title': 'movie'}, inplace=True)

In [20]:
cleaned_movies_ratings_df = cleaned_movies_ratings_df[(cleaned_movies_ratings_df['director_rating'] >= 7.0) & 
                                                     (cleaned_movies_ratings_df['nummovies'] >= 3)] 

In [21]:
cleaned_movies_ratings_df['primary_name'].value_counts(
)

Steven Spielberg         7
Denis Villeneuve         6
James Wan                6
David Yates              5
Richard Linklater        5
                        ..
Alejandro G. Iñárritu    3
Luca Guadagnino          3
Werner Herzog            3
Oriol Paulo              3
Ben Affleck              3
Name: primary_name, Length: 88, dtype: int64

In [22]:
cleaned_movies_ratings_df.drop(columns=['movie_id', 'original_title', 'start_year', 'runtime_minutes', 
                                       'person_id', 'numvotes', 'birth_year'], inplace=True)

In [23]:
cleaned_movies_ratings_df.to_csv('data/final_directors.csv')

In [24]:
q = """

SELECT *
FROM movie_basics
WHERE primary_title == 'The Dark Knight Rises'
;
"""

pd.read_sql(q, im_db_conn)

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt1345836,The Dark Knight Rises,The Dark Knight Rises,2012,164.0,"Action,Thriller"


In [25]:
q = """

SELECT DISTINCT *
FROM movie_basics
LEFT JOIN writers
    USING(movie_id)
LEFT JOIN movie_ratings
    USING(movie_id)
LEFT JOIN persons
    USING(person_id)
WHERE numvotes > 9999 and averagerating != 'NaN'
;
"""

cleaned_movies_writers_df = pd.read_sql(q, im_db_conn)

In [26]:
writer_series = cleaned_movies_writers_df.groupby('primary_name')['averagerating'].mean()

In [27]:
writer_ratings_df = pd.DataFrame(series) 

In [28]:
writer_ratings_df = writer_ratings_df.rename(columns={'averagerating':'writer_rating'})

In [29]:
writer_ratings_df = writer_ratings_df.reset_index()
writer_ratings_df.head()

,primary_name,writer_rating
0,A.R. Murugadoss,7.600
1,Aanand L. Rai,6.925
2,Aaron Hann,6.000
3,Aaron Harvey,4.600
4,Aaron Horvath,6.800


In [30]:
cleaned_movies_writers_df = cleaned_movies_writers_df.merge(writer_ratings_df, how='left', on='primary_name')

In [31]:
cleaned_movies_writers_df.loc[cleaned_movies_writers_df['primary_name'] == 'Aaron Hann']

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession,writer_rating
5276,tt3118452,Circle,Circle,2015,87.0,"Drama,Mystery,Sci-Fi",nm5278146,6.0,30645,Aaron Hann,NaN,NaN,"director,writer,cinematographer",6.0


In [32]:
nummovies1_series = cleaned_movies_writers_df['primary_name'].value_counts()
nummovies1_df = pd.DataFrame(nummovies1_series)

In [33]:
nummovies1_df = nummovies1_df.reset_index()

In [34]:
nummovies1_df = nummovies1_df.rename(columns={'index':'primary_name', 'primary_name': 'nummovies'})

In [35]:
nummovies1_df

,primary_name,nummovies
0,Jack Kirby,22
1,Stan Lee,21
2,Luc Besson,12
3,Nicholas Stoller,11
4,Steven Knight,10
...,...,...
4313,Josh Boone,1
4314,Joyce Chan,1
4315,Kangana Ranaut,1
4316,Jonathan Tropper,1


In [36]:
cleaned_movies_writers_df = cleaned_movies_writers_df.merge(nummovies1_df, how='left', on='primary_name')

In [37]:
cleaned_movies_writers_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,person_id,averagerating,numvotes,primary_name,birth_year,death_year,primary_profession,writer_rating,nummovies
0,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm0001317,4.2,50352,Renny Harlin,1959.0,NaN,"producer,director,writer",5.3,1.0
1,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm0316417,4.2,50352,Daniel Giat,1955.0,NaN,"writer,miscellaneous",NaN,1.0
2,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm0393517,4.2,50352,Sean Hood,1966.0,NaN,"writer,art_department,producer",NaN,2.0
3,tt1043726,The Legend of Hercules,The Legend of Hercules,2014,99.0,"Action,Adventure,Fantasy",nm1048866,4.2,50352,Giulio Steve,NaN,NaN,"producer,production_manager,writer",NaN,1.0
4,tt1210166,Moneyball,Moneyball,2011,133.0,"Biography,Drama,Sport",nm0001873,7.6,326657,Steven Zaillian,1953.0,NaN,"writer,producer,director",NaN,3.0


In [38]:
cleaned_movies_writers_df = cleaned_movies_writers_df[(cleaned_movies_writers_df['writer_rating'] >= 7.0) & 
                                                     (cleaned_movies_writers_df['nummovies'] >= 3)] 

In [39]:
cleaned_movies_writers_df['primary_name'].value_counts()

Joel Coen                8
Ethan Coen               8
Anurag Kashyap           7
Christopher McQuarrie    7
Adam McKay               7
                        ..
Andrey Zvyagintsev       3
Roman Polanski           3
Jacques Audiard          3
Nuri Bilge Ceylan        3
Taika Waititi            3
Name: primary_name, Length: 75, dtype: int64

In [40]:
cleaned_movies_writers_df.drop(columns=['movie_id', 'original_title', 'start_year', 'runtime_minutes', 
                                       'person_id', 'numvotes', 'birth_year'], inplace=True)

In [41]:
cleaned_movies_writers_df.to_csv('data/final_writers.csv')

In [42]:
# budgets_df = pd.read_csv('data/tn.movie_budgets.csv')

In [43]:
# budgets_df.info()

The below is filtering our data to only include movies with 10000 votes on IMDB, representing a healthy audience level. $$$$$$$

In [44]:
# q = """
# SELECT numvotes, primary_title
# FROM movie_ratings
#     LEFT JOIN movie_basics
#         USING(movie_id)
# WHERE numvotes > 9999
# ;
# """

# pd.read_sql(q, im_db_conn)

In [45]:
# q = """

# SELECT typeof(person_id)
# FROM writers

# ;
# """

# pd.read_sql(q, im_db_conn)

Top Writers table

In [46]:
# q = """
# SELECT persons.person_id, persons.primary_name, AVG(movie_ratings.averagerating) AS writer_rating
# FROM persons
# LEFT JOIN writers 
#     USING(person_id)
# LEFT JOIN movie_basics
#     Using(movie_id)
# LEFT JOIN movie_ratings
#     Using(movie_id)
# WHERE numvotes > 9999 and averagerating != 'NaN'
# GROUP BY persons.person_id, persons.primary_name
# HAVING writer_rating > 7 and COUNT(writers.movie_id) > 5
# ORDER BY writer_rating DESC
# """

# writers_df = pd.read_sql(q, im_db_conn)
# writers_df.to_csv('data/writers.csv', index=False)
# writers_df.head()

Top Directors table

In [47]:
# q = """
# SELECT persons.person_id, persons.primary_name, AVG(movie_ratings.averagerating) AS director_rating, COUNT(persons.person_id) as nummovies
# FROM persons
# LEFT JOIN directors
#     USING(person_id)
# LEFT JOIN movie_basics
#     USING(movie_id)
# LEFT JOIN movie_ratings
#     USING(movie_id)
# WHERE numvotes > 9999 and averagerating != 'NaN'
# GROUP BY persons.primary_name
# HAVING director_rating > 7 AND nummovies > 5
# ORDER BY director_rating DESC
# """

# director_ratings_df = pd.read_sql(q, im_db_conn)
# director_ratings_df.to_csv('data/directors.csv', index=False)
# director_ratings_df.head(30)

In [48]:
 q = """
SELECT DISTINCT *
FROM movie_basics
LEFT JOIN movie_ratings
    USING(movie_id)
 ;
 """

genres_df = pd.read_sql(q, im_db_conn)
genres_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama,6.9,4517.0
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama",6.1,13.0
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy",6.5,119.0


In [49]:
# q = """

# SELECT primary_title as movie, genres, averagerating
# FROM movie_basics
# LEFT JOIN movie_ratings
#     USING(movie_id)
# ;
# """

# genres_df = pd.read_sql(q, im_db_conn)
# genres_df.head()

In [50]:
# list(genres_df['genres'][0].split(','))


In [51]:
genres_df.dropna(subset=['genres'], inplace=True)
genres_df.dropna(subset=['averagerating'], inplace=True)

In [52]:
#print(genres_df)

In [53]:
genres_df['genre_list'] = genres_df['genres'].map(lambda x: list(x.split(',')))

In [54]:
print(genres_df['genre_list'])

0           [Action, Crime, Drama]
1               [Biography, Drama]
2                          [Drama]
3                  [Comedy, Drama]
4         [Comedy, Drama, Fantasy]
                    ...           
146114               [Documentary]
146115               [Documentary]
146122             [Drama, Family]
146125               [Documentary]
146134               [Documentary]
Name: genre_list, Length: 73052, dtype: object


In [55]:
genres_df = genres_df.explode('genre_list')

In [56]:
genres_df['genre_list'].value_counts()

Drama          30788
Documentary    17753
Comedy         17290
Thriller        8217
Horror          7674
Action          6988
Romance         6589
Crime           4611
Adventure       3817
Biography       3809
Family          3412
Mystery         3039
History         2825
Sci-Fi          2206
Fantasy         2126
Music           1968
Animation       1743
Sport           1179
War              853
Musical          721
News             579
Western          280
Reality-TV        17
Adult              3
Game-Show          2
Short              1
Name: genre_list, dtype: int64

In [57]:
genres_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres,averagerating,numvotes,genre_list
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Action
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Crime
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama",7.0,77.0,Drama
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,Biography
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama",7.2,43.0,Drama


In [58]:
budgets_df = pd.read_csv("data/tn.movie_budgets.csv")
budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [59]:
movies_df = pd.read_csv("data/tmdb.movies.csv")
movies_df.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [60]:
movies_df.rename(columns = {'original_title':'movie'},inplace = True)

In [61]:
genres_df.rename(columns = {'original_title':'movie'},inplace = True)

In [62]:
movies_and_budgets_df = movies_df.merge(budgets_df, how='inner', on = 'movie')
movies_and_budgets_df.head()

,Unnamed: 0,genre_ids,id_x,original_language,movie,popularity,release_date_x,title,vote_average,vote_count,id_y,release_date_y,production_budget,domestic_gross,worldwide_gross
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,30,"Mar 26, 2010","$165,000,000","$217,581,232","$494,870,992"
1,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,15,"May 7, 2010","$170,000,000","$312,433,331","$621,156,389"
2,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995","$30,000,000","$191,796,233","$364,545,516"
3,2473,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,37,"Nov 22, 1995","$30,000,000","$191,796,233","$364,545,516"
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,38,"Jul 16, 2010","$160,000,000","$292,576,195","$835,524,642"


In [63]:
#Convert strings to ints
def clean_and_convert_to_int(dataframe, columns_to_clean):
    for col in columns_to_clean:
        dataframe[col] = dataframe[col].str.replace(r'[\$,\D]', "", regex=True)
        dataframe[col] = pd.to_numeric(dataframe[col], errors='coerce')
        dataframe[col] = dataframe[col].astype(int)  # Use 'Int64' to handle NaN as nullable integers
    return dataframe

# List of columns to clean and convert to integers
movies_and_budgets_columns = ['production_budget', 'domestic_gross', 'worldwide_gross']
#rotten_tomatoes_columns = ['box_office', 'runtime']

# Call the function for each DataFrame
movies_and_budgets_df = clean_and_convert_to_int(movies_and_budgets_df, movies_and_budgets_columns)
#rotten_tomatoes_df = clean_and_convert_to_int(rotten_tomatoes_df, rotten_tomatoes_columns)

In [64]:
movies_budgets_genres_df = movies_and_budgets_df.merge(genres_df, how='inner', on = 'movie')
movies_budgets_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6721 entries, 0 to 6720
Data columns (total 23 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         6721 non-null   int64  
 1   genre_ids          6721 non-null   object 
 2   id_x               6721 non-null   int64  
 3   original_language  6721 non-null   object 
 4   movie              6721 non-null   object 
 5   popularity         6721 non-null   float64
 6   release_date_x     6721 non-null   object 
 7   title              6721 non-null   object 
 8   vote_average       6721 non-null   float64
 9   vote_count         6721 non-null   int64  
 10  id_y               6721 non-null   int64  
 11  release_date_y     6721 non-null   object 
 12  production_budget  6721 non-null   int64  
 13  domestic_gross     6721 non-null   int64  
 14  worldwide_gross    6721 non-null   int64  
 15  movie_id           6721 non-null   object 
 16  primary_title      6721 

In [65]:
movies_budgets_genres_df.drop(columns= ['genre_ids', 'Unnamed: 0', 'original_language', 'release_date_x', 'title',
                                        'vote_count', 'vote_average', 'id_y', 'release_date_y', 'id_x', 'start_year', 'runtime_minutes', 'primary_title', 'genres'], inplace=True)

In [66]:
movies_budgets_genres_df = movies_budgets_genres_df[(movies_budgets_genres_df['numvotes'] > 9999) & 
                                                    (movies_budgets_genres_df['averagerating'] != 'NaN')]

In [67]:
movies_budgets_genres_df.shape

(4257, 9)

In [68]:
genre_series = movies_budgets_genres_df.groupby('genre_list')['averagerating'].mean()

In [69]:
genre_rating_df = pd.DataFrame(genre_series) 

In [70]:
genre_rating_df = genre_rating_df.reset_index()

In [71]:
genre_rating_df = genre_rating_df.rename(columns = {'averagerating': 'genre_rating'})

In [72]:
movies_budgets_genres_df = movies_budgets_genres_df.merge(genre_rating_df, how='inner', on = 'genre_list')
movies_budgets_genres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4257 entries, 0 to 4256
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              4257 non-null   object 
 1   popularity         4257 non-null   float64
 2   production_budget  4257 non-null   int64  
 3   domestic_gross     4257 non-null   int64  
 4   worldwide_gross    4257 non-null   int64  
 5   movie_id           4257 non-null   object 
 6   averagerating      4257 non-null   float64
 7   numvotes           4257 non-null   float64
 8   genre_list         4257 non-null   object 
 9   genre_rating       4257 non-null   float64
dtypes: float64(4), int64(3), object(3)
memory usage: 365.8+ KB


In [73]:
movies_budgets_genres_df.head()

,movie,popularity,production_budget,domestic_gross,worldwide_gross,movie_id,averagerating,numvotes,genre_list,genre_rating
0,How to Train Your Dragon,28.734,165000000,217581232,494870992,tt0892769,8.1,611299.0,Action,6.384407
1,Iron Man 2,28.515,170000000,312433331,621156389,tt1228705,7.0,657690.0,Action,6.384407
2,Inception,27.920,160000000,292576195,835524642,tt1375666,8.8,1841066.0,Action,6.384407
3,Megamind,22.855,130000000,148415853,321887208,tt1001526,7.3,207488.0,Action,6.384407
4,The Expendables,21.517,82000000,103068524,268268174,tt1320253,6.5,307897.0,Action,6.384407


In [74]:
movies_budgets_genres_df.to_csv('data/final_movies_budgets_genres.csv')

In [75]:
budgets_directors_df = movies_and_budgets_df.merge(cleaned_movies_ratings_df, how='inner', on = 'movie')
budgets_directors_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          226 non-null    int64  
 1   genre_ids           226 non-null    object 
 2   id_x                226 non-null    int64  
 3   original_language   226 non-null    object 
 4   movie               226 non-null    object 
 5   popularity          226 non-null    float64
 6   release_date_x      226 non-null    object 
 7   title               226 non-null    object 
 8   vote_average        226 non-null    float64
 9   vote_count          226 non-null    int64  
 10  id_y                226 non-null    int64  
 11  release_date_y      226 non-null    object 
 12  production_budget   226 non-null    int64  
 13  domestic_gross      226 non-null    int64  
 14  worldwide_gross     226 non-null    int64  
 15  genres              226 non-null    object 
 16  averager

In [76]:
budgets_directors_df.head()

,Unnamed: 0,genre_ids,id_x,original_language,movie,popularity,release_date_x,title,vote_average,vote_count,...,production_budget,domestic_gross,worldwide_gross,genres,averagerating,primary_name,death_year,primary_profession,director_rating,nummovies
0,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,...,165000000,217581232,494870992,"Action,Adventure,Animation",8.1,Dean DeBlois,NaN,"writer,animation_department,director",7.833333,3.0
1,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,...,160000000,292576195,835524642,"Action,Adventure,Sci-Fi",8.8,Christopher Nolan,NaN,"writer,producer,director",8.425000,4.0
2,8,"[16, 10751, 35]",20352,en,Despicable Me,23.673,2010-07-09,Despicable Me,7.2,10057,...,69000000,251513985,543464573,"Animation,Comedy,Family",7.7,Chris Renaud,NaN,"producer,actor,director",7.000000,4.0
3,19,"[18, 53, 9648]",11324,en,Shutter Island,18.060,2010-02-18,Shutter Island,8.1,12625,...,80000000,128012934,299461782,"Mystery,Thriller",8.1,Martin Scorsese,NaN,"producer,director,actor",7.750000,4.0
4,21,"[28, 35]",37834,en,Knight and Day,17.713,2010-06-23,Knight and Day,6.0,2494,...,117000000,76423035,258751370,"Action,Adventure,Comedy",6.3,James Mangold,NaN,"producer,director,writer",7.033333,3.0


In [77]:
budgets_directors_df.drop(columns = ['genre_ids', 'Unnamed: 0', 'original_language', 'release_date_x', 'title',
                                        'vote_count', 'vote_average', 'id_y', 'release_date_y', 'id_x', 'genres', 
                                     'primary_profession', 'nummovies'], inplace=True)

In [98]:
budgets_directors_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226 entries, 0 to 225
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   movie              226 non-null    object 
 1   popularity         226 non-null    float64
 2   production_budget  226 non-null    int64  
 3   domestic_gross     226 non-null    int64  
 4   worldwide_gross    226 non-null    int64  
 5   averagerating      226 non-null    float64
 6   primary_name       226 non-null    object 
 7   death_year         0 non-null      float64
 8   director_rating    226 non-null    float64
dtypes: float64(4), int64(3), object(2)
memory usage: 17.7+ KB


In [99]:
budgets_directors_df.to_csv('data/budgets_directors.csv')

In [79]:
# q = """
# SELECT genres, AVG(movie_ratings.averagerating) AS genre_rating
# FROM movie_basics
# LEFT JOIN movie_ratings
#     USING(movie_id)
# WHERE numvotes > 9999 and averagerating != 'NaN'
# GROUP BY movie_basics.genres
# HAVING COUNT(genres) > 20
# ORDER BY genre_rating DESC
# """

# genres_ratings_df = pd.read_sql(q, im_db_conn)
# print(genres_ratings_df)

In [80]:
# q = """

# SELECT *, AVG(averagerating) As avg_rating
# FROM movie_basics
# LEFT JOIN movie_ratings
#     USING (movie_id)
#     LEFT JOIN writers
#         USING(movie_id)
#             LEFT JOIN persons
#                 USING(person_id)
# GROUP BY writers.person_id
# HAVING avg_rating != 'NaN'
# ORDER BY avg_rating
# ;
# """

# pd.read_sql(q, im_db_conn)

In [81]:
# q = """

# SELECT *
# FROM persons
# LEFT JOIN directors
#     USING (person_id)
#     LEFT JOIN writers
#         USING(person_id)
# WHERE primary_profession LIKE '%director%' or primary_profession LIKE '%writer%'
# GROUP BY person_id
# ;
# """

# pd.read_sql(q, im_db_conn)

In [82]:
# df = pd.read_sql(q, im_db_conn)

In [83]:
#sns.scatterplot(data=df, x = 'genres', y='averagerating')
#plt.xlabel('averagerating')
#plt.ylabel('numvotes')
#plt.show()

In [84]:
#plt.plot(df['genres'], df['averagerating'])
#plt.xlabel('averagerating')
#plt.ylabel('numvotes')
#plt.title('Plot of Data from SQL Database')
#plt.show()

In [85]:
# q = """

# SELECT *
# FROM persons
# WHERE primary_profession LIKE '%director%' OR primary_profession LIKE '%writer%'
# GROUP BY Primary_name
# ;
# """

# pd.read_sql(q, im_db_conn)

In [86]:
# q = """

# SELECT *
# FROM movie_basics

# ;
# """

# pd.read_sql(q, im_db_conn)

In [87]:
#q = """
#SELECT numvotes, primary_title, averagerating
#FROM movie_ratings
#    LEFT JOIN movie_basics
#        USING(movie_id)
#WHERE numvotes > 9999
#;
#"""
#pd.read_sql(q, im_db_conn)
#movie_ratings_df.to_csv('data/imdb_movie_ratings')

In [88]:
movies_and_budgets_df.to_csv('data/movies_and_budgets.csv', index=False)

In [89]:
#anova_genre = ols('worldwide_gross ~C(genre_list)', data=movies_budgets_genres_df).fit()
#anova_genre.summary()

In [90]:
#movies_budgets_genres_df.info()

In [91]:
#q = """
#SELECT *, primary_title as movie
#FROM movie_basics
#;
#"""
#basics_df = pd.read_sql(q, im_db_conn)
#basics_df.head()

In [92]:
#directors_df['person_id'].value_counts()

In [93]:
#final_directors_df = directors_df.merge(director_ratings_df, how='inner', on = 'person_id')
#final_directors_df.dropna(subset = ['director_rating'], inplace=True)


In [94]:
#directors_df['primary_name'].value_counts()

In [95]:
#director_counts = final_directors_df['primary_name_x'].value_counts()
#pop_directors = director_counts.loc[director_counts > 4].index.tolist()
#final_directors_df = final_directors_df[final_directors_df['primary_name_x'].isin(pop_directors)]

In [96]:
#final_directors_df['primary_name_x'].value_counts()